# DS 4400 Final Project
## Devanagari Handwritten Character Classification

### Eirean Co, Levi Kaplan, Justine Luo



In [1]:
import pandas as pd
import numpy as np


import os
from PIL import Image

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose

import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

In [ ]:
"""
    Takes the DevanagariHandwrittenCharacter dataset and turns it into a 
        Pandas DataFrame, with the values of each pixel for each image from 0 to 255 
        as the first 2024 columns and the corresponding character name as the final column.
"""
feature_list = list()
parent_directory = "DevanagariHandwrittenCharacterDataset/"
# loops through directory, getting the test and train folders
for parent_folder in os.listdir(parent_directory):
    child_directory = os.path.join(parent_directory, parent_folder)
    # goes through test and train folders
    for child_folder in os.listdir(child_directory):
        # gets the name of the character that this folder is for
        split_str = child_folder.split("_")
        current_character = split_str[-1]
        # goes through each image in the folder and adds it to the feature list
        for img in os.listdir(os.path.join(child_directory, child_folder)):
            img_directory = os.path.join(child_directory, child_folder)
            if img.endswith(".png"):
                imframe = Image.open(os.path.join(img_directory, img))
                npframe = np.array(imframe.getdata())
                npframe_with_char = np.append(npframe, current_character)
                feature_list.append(npframe_with_char)
# turn feature list into an array
feature_array = np.asarray(feature_list)
# create a dataframe for the array
feature_df = pd.DataFrame(feature_array)
# save the df as a csv
feature_df.to_csv("devanagari_df.csv")
feature_df

In [6]:
# Load the dataset
df = pd.read_csv(r'devanagari_df.csv')
df = df.dropna()

# define the features
features = df.iloc[:, 1:1025]
features = features.astype('int64')

# define the target, using consistent types (as some targets are initially ints)
target = df.iloc[:, 1025]
target = target.astype(str)
features

,0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
91996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
91997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
91998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# get the train test split
X_train, X_test, y_train, y_test = train_test_split(features, target, random_state=3000)

### Naive Bayes

In [ ]:
# fit a gaussian Naive Bayes to the data
gnb = GaussianNB()
gnb.fit(X_train, y_train)

GaussianNB()

In [ ]:
predicted_test = gnb.predict(X_test)
predicted_train = gnb.predict(X_train)

In [ ]:
acc_test = accuracy_score(predicted_test, y_test)
print('test accuracy', acc_test)
acc_train = accuracy_score(predicted_train, y_train)
print('train accuracy', acc_train)

test accuracy 0.4715086646279307
train accuracy 0.4776758409785933


In [ ]:
# define search
params_NB = {'var_smoothing': np.logspace(0,-9, num=10)}
search = GridSearchCV(gnb, params_NB, scoring='accuracy')
search.fit(X_train, y_train)
model = search.best_estimator_ # GaussianNB(var_smoothing=0.1)
print(model)
scores = cross_val_score(model, features, target, scoring='accuracy')
# report performance
print('Accuracy: %.3f Std: (%.3f)' % (np.mean(scores), np.std(scores)))

GaussianNB(var_smoothing=0.1)
Accuracy: 0.529 (0.001)


In [ ]:
# optimal smoothing found via grid search 
optimal_nb_model = search.best_estimator_
opt_nb_predicted_test = optimal_nb_model.predict(X_test)
opt_nb_predicted_train = optimal_nb_model.predict(X_train)
print(classification_report(y_test, opt_nb_predicted_test))
print(classification_report(y_train, opt_nb_predicted_train))

              precision    recall  f1-score   support

           0       0.76      0.97      0.85        76
           1       0.31      0.88      0.46        58
           2       0.67      0.60      0.64       518
           3       0.70      0.45      0.55       486
           4       0.65      0.81      0.72       494
           5       0.72      0.78      0.75       507
           6       0.57      0.68      0.62        91
           7       0.37      0.78      0.51        74
           8       0.15      0.84      0.26        77
           9       0.21      0.66      0.31        65
        adna       0.65      0.50      0.56       505
          ba       0.52      0.33      0.40       498
         bha       0.79      0.50      0.61       512
         cha       0.57      0.67      0.62       500
        chha       0.76      0.37      0.49       495
       chhya       0.38      0.76      0.51       509
          da       0.58      0.43      0.49       517
         daa       0.79    

In [ ]:
print('accuracy score on test with opt model', accuracy_score(opt_nb_predicted_test, y_test))
print('accuracy score on train with opt model', accuracy_score(opt_nb_predicted_train, y_train))


accuracy score on test with opt model 0.5352191641182467
accuracy score on train with opt model 0.5377505946313286


### SVM

In [ ]:
def svc_train_test_report(model, X_train, y_train, X_test, y_test):
  model = model.fit(X_train, y_train)
  y_pred_train_svc = model.predict(X_train)
  train_score_svc = accuracy_score(y_train, y_pred_train_svc)
  print("Training accuracy: ", train_score_svc)
  print(classification_report(y_train, y_pred_train_svc))

  y_pred_test_svc = model.predict(X_test)
  test_score_svc = accuracy_score(y_test, y_pred_test_svc)
  print("Testing accuracy: ", test_score_svc)
  print(classification_report(y_test, y_pred_test_svc))

  return model


svc = svc_train_test_report(SVC(), X_train, y_train, X_test, y_test)

Training accuracy:  0.9823562891291975
              precision    recall  f1-score   support

         bha       1.00      0.98      0.99       221
         cha       0.98      0.99      0.99       226
        chha       1.00      1.00      1.00       219
         gha       0.97      0.98      0.98       224
   patalosaw       0.98      1.00      0.99       229
         pha       0.98      0.98      0.98       178
        thaa       0.99      0.98      0.98       228
         waw       0.97      0.96      0.96       232

    accuracy                           0.98      1757
   macro avg       0.98      0.98      0.98      1757
weighted avg       0.98      0.98      0.98      1757

Testing accuracy:  0.9112627986348123
              precision    recall  f1-score   support

         bha       0.94      0.95      0.94        79
         cha       0.88      0.93      0.91        74
        chha       0.97      0.94      0.96        81
         gha       0.85      0.89      0.87        76
 

In [ ]:
def svc_tune(X_train, y_train):
  hp_grid = {'kernel': ['linear', 'poly', 'rbf'], 'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001, 0.0001]}
  svc_grid = GridSearchCV(SVC(), hp_grid, refit=True, verbose=3)
  svc_grid.fit(X_train,y_train)
  return svc_grid

svc_grid = svc_tune(X_train, y_train)

Fitting 5 folds for each of 60 candidates, totalling 300 fits
[CV 1/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.858 total time=   0.5s
[CV 2/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.810 total time=   0.5s
[CV 3/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.858 total time=   0.5s
[CV 4/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.795 total time=   0.5s
[CV 5/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.849 total time=   0.5s
[CV 1/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.898 total time=   0.7s
[CV 2/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.852 total time=   0.7s
[CV 3/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.909 total time=   0.7s
[CV 4/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.860 total time=   0.7s
[CV 5/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.897 total time=   0.6s
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.131 total time=   1.5s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf

In [ ]:
tuned_svc = svc_grid.best_estimator_
print(tuned_svc.get_params())

{'C': 0.1, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 1, 'kernel': 'linear', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}


In [ ]:
svc_train_test_report(tuned_svc, X_train, y_train, X_test, y_test)

Training accuracy:  1.0
              precision    recall  f1-score   support

         cha       1.00      1.00      1.00        66
         waw       1.00      1.00      1.00       229

    accuracy                           1.00       295
   macro avg       1.00      1.00      1.00       295
weighted avg       1.00      1.00      1.00       295

Testing accuracy:  0.9090909090909091
              precision    recall  f1-score   support

         cha       0.91      0.75      0.82        28
         waw       0.91      0.97      0.94        71

    accuracy                           0.91        99
   macro avg       0.91      0.86      0.88        99
weighted avg       0.91      0.91      0.91        99



SVC(C=0.1, gamma=1, kernel='linear')

Neural Networks

In [44]:
# import the csv as a pandas dataframe 
df = pd.read_csv("devanagari_df.csv", index_col=0)
df

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1025) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [46]:
"""
    Defines the Feed Forward Neural Network Module.
    This module takes in the number of hidden layers, the nodes per layer, the training data, and the activation function,
        and outputs the model to be trained using its forward method.
    Args:
        num_hidden_layers (Int): the number of hidden layers of the neural network (doesn't include the input/output layers)
        nodes_per_layer (List[Int]): the number of nodes per layer. The last one should be 1, and the length should be (2 + num_hidden_layers)
        activation_function (Function): Must be one of torch.nn.Sigmoid(), torch.nn.ReLU(), torch.tanh(), torch.nn.Identity()
    Returns:
        The Feed Forward Neural Network model to be trained
"""
class Feed_Forward(torch.nn.Module):
    def __init__(self, num_hidden_layers, nodes_per_layer, activation_function):
        super(Feed_Forward, self).__init__()
        # assert Module inputs adhere to constraints
        assert(num_hidden_layers == len(nodes_per_layer) - 2)

        self.num_hidden_layers = num_hidden_layers
        self.nodes_per_layer = nodes_per_layer
        self.activation_function = activation_function
        self.layers = []
        # define the first layer as going from the initial data count to the first hidden layer's node count
        self.fc1 = torch.nn.Linear(nodes_per_layer[0], nodes_per_layer[1])
        # Append the first layer and activation function to this Module
        self.layers.append(self.fc1)
        self.layers.append(activation_function)

        # Append layers to the model for each hidden layer
        for i in range(num_hidden_layers - 1):
            self.layers.append(torch.nn.Linear(self.nodes_per_layer[i + 1], self.nodes_per_layer[i + 2]))
            self.layers.append(activation_function)

        # Append the output layer to the model
        self.output_layer = torch.nn.Linear(self.nodes_per_layer[-2], self.nodes_per_layer[-1])
        self.output_activation_sigmoid = torch.nn.Sigmoid()
        self.layers.append(self.output_layer)
        self.layers.append(self.output_activation_sigmoid)

    # runs a step through the model, generating the output from this epoch of training
    def forward(self, x):
        model = torch.nn.Sequential(*self.layers)
        return model.forward(x)

In [47]:
def train_model(model, epochs, X_trn, Y_trn):
    """
        Trains the given model, performing the given number of epochs over the given Training and Testing data.
        Args:
            model (torch.nn.Module): The Feed Forward Neural Network to be trained
            epochs (int): The total number of epochs to train over
            X_trn (Tensor): the training features
            Y_trn (Tensor): the training target
    """
    # define a loss criteria for the model (use Cross-Entropy Loss, as this is a classification problem)
    criterion = torch.nn.CrossEntropyLoss()
    # define an optimizer for the model (use Stochastic Gradient Descent)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)
    # set the model to train
    model.train()
    # set default value for final layer output
    final_layer_output = 0
    # for every epoch, train the model
    for epoch in range(epochs):
        optimizer.zero_grad()
        # Move forward through the model
        y_pred = model(X_trn)
        #y_pred = y_pred.type(torch.LongTensor)

        # if this is the last epoch, we save the predictions
        if(epoch == epochs - 1):
            final_layer_output = y_pred.round()
        # Compute the loss
        loss = criterion(y_pred, Y_train)
        # Back-propagate to adjust weights
        loss.backward()
        # Step through model
        optimizer.step()
    return model, final_layer_output


In [48]:
def create_and_train_ff_nn(num_hidden_layers, nodes_per_layer, X_trn, Y_trn, activation_function, epochs):
    """
        Creates a Feed Forward Neural Network, and then trains it over the specified number of epochs
        Args:
            num_hidden_layers (Int): the total number of hidden layers for the neural network, either 1 or 2
            nodes_per_layer (List[Int]): the number of nodes for each layer of the neural network
            X_trn, Y_trn (Tensor): the training data 
            activation_function (Function): the activation function used
            epochs (Int): the number of epochs to train over
        Returns:
            the trained model

    """
    # define the feed forward model
    model = Feed_Forward(num_hidden_layers, nodes_per_layer, activation_function)
    # train the model and get the final output
    model, final_layer_output = train_model(model, epochs, X_trn, Y_trn)
    return model

In [49]:
def test_model(model, X_trn, Y_trn, X_tst, Y_tst):
    # testing, so don't want to use a gradient
    with torch.no_grad():
        # testing the model
        model.eval()
        # make the predictions
        y_pred = model(X_trn)
        y_pred = torch.argmax(y_pred, dim=-1).cpu().detach().numpy()
        # find the accuracy for training and testing
        training_accuracy = accuracy_score(Y_trn, y_pred)
        y_pred = model(X_tst)
        y_pred = torch.argmax(y_pred, dim=-1).cpu().detach().numpy()
        testing_accuracy = accuracy_score(Y_tst, y_pred)
        return training_accuracy, testing_accuracy

In [50]:
features = df.iloc[:, :-1]
target = df.iloc[:, -1]
target = [str(x) for x in target]
label_encoder = LabelEncoder()
target = label_encoder.fit_transform(target)

In [51]:
nodes_per_layer_list = [[X_train.shape[1], 100, 46], [X_train.shape[1], 100, 100, 46], [X_train.shape[1], 500, 100, 46], [X_train.shape[1], 100, 100, 100, 46], [X_train.shape[1], 100, 100, 100, 100, 46]]
hidden_layers_list  = [len(x) - 2 for x in nodes_per_layer_list]

In [ ]:
epochs = 20
n_splits = 5
activation_functions_list = [torch.nn.ReLU(), torch.nn.Sigmoid(), torch.nn.Tanh()]
kf = StratifiedKFold(n_splits=n_splits, random_state=4400, shuffle=True)
for a in activation_functions_list:
  for i, nodes_per_layer in enumerate(nodes_per_layer_list):
      num_hidden_layers = hidden_layers_list[i]
      training_count = 0.0
      testing_count = 0.0
      print("Running model with ", num_hidden_layers, "hidden layers: ", nodes_per_layer)
      for train_index, test_index in kf.split(features, target):
          X_train, X_test = features[train_index[0]:train_index[-1]], features[test_index[0]:test_index[-1]]
          Y_train, Y_test = target[train_index[0]:train_index[-1]], target[test_index[0]:test_index[-1]]
          X_train = X_train.to_numpy()
          X_test = X_test.to_numpy()
          # turn the training and testing data into tensors
          X_train = torch.FloatTensor(X_train)
          Y_train = torch.LongTensor(Y_train)
          X_test = torch.FloatTensor(X_test)
          Y_test = torch.LongTensor(Y_test)

          #for a in activation_functions_list:
          model = create_and_train_ff_nn(num_hidden_layers, nodes_per_layer, X_train, Y_train, a, epochs)
          curr_training_count, curr_testing_count = test_model(model, X_train, Y_train, X_test, Y_test)
          training_count = training_count + curr_training_count
          testing_count = testing_count + curr_testing_count
      training_accuracy = training_count / n_splits
      testing_accuracy = testing_count / n_splits
      print("\tAverage training accuracy: ", training_accuracy)
      print("\tAverage testing accuracy: ", testing_accuracy, "\n")

Running model with  1 hidden layers:  [1024, 100, 46]
	Average training accuracy:  0.6720943043113902
	Average testing accuracy:  0.6720869427530742 

Running model with  2 hidden layers:  [1024, 100, 100, 46]
